In [85]:
import Titanic
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_val_score,train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline


In [42]:
df = pd.read_csv('test.csv')
df['Title'] = df['Name'].str.extract(r',\s*([^\.]*)\.', expand=False)
title_mapping = {
        'Mr': 'Mr',
        'Mrs': 'Mrs',
        'Miss': 'Miss',
        'Master': 'Master',
        'Mlle': 'Miss',     # French for Miss
        'Ms': 'Miss',
        'Mme': 'Mrs',       # French for Mrs
        'Lady': 'Mrs',
        'Countess': 'Mrs',
        'Dona': 'Mrs',
        'Sir': 'Mr',
        'Jonkheer': 'Mr',
        'Don': 'Mr',
        'Rev': 'Mr',
        'Dr': 'Mr',
        'Col': 'Mr',
        'Major': 'Mr',
        'Capt': 'Mr'
        }
df['Title'] = df['Title'].map(title_mapping).fillna('Mrs')
df['Deck'] = df['Cabin'].str[0]
df['Deck'] = df['Deck'].replace('T', 'U')
df['Deck'] = df['Deck'].fillna('U')
df = df.drop(['Cabin'], axis=1)

In [46]:

df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 18, 35, 60, 80], labels=['Child', 'Teen', 'YoungAdult', 'Adult', 'Senior'])
df = df.dropna(subset=['Fare'])
df.isnull().sum()

PassengerId     0
Pclass          0
Name            0
Sex             0
Age            86
SibSp           0
Parch           0
Ticket          0
Fare            0
Embarked        0
Title           0
Deck            0
AgeGroup       86
dtype: int64

In [80]:
features = ['Title']
known = df[df['AgeGroup'].notnull()]
unknown = df[df['AgeGroup'].isnull()]
encode = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
encode.fit(df[features])
X = encode.transform(known[['Title']])
y = known['AgeGroup']

0      YoungAdult
1           Adult
2          Senior
3      YoungAdult
4      YoungAdult
          ...    
409         Child
411         Adult
412    YoungAdult
414         Adult
415         Adult
Name: AgeGroup, Length: 331, dtype: category
Categories (5, object): ['Child' < 'Teen' < 'YoungAdult' < 'Adult' < 'Senior']

In [98]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5)
#cross_val_score(knn, X, y, cv=10, scoring='accuracy').mean()
grid = GridSearchCV(knn, 
                    param_grid = {'n_neighbors': range(1,20),
                                 'weights' : ["uniform","distance"]}, 
                    cv = 10,
                   scoring='accuracy')
grid.fit(X,y)
pd.DataFrame(grid.cv_results_)
grid.best_estimator_
grid.best_score_
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
# knn.fit(X_train, y_train)
# y_pred = knn.predict(X_test)
# metrics.accuracy_score(y_test, y_pred)

np.float64(0.5829768270944741)

In [111]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(solver='lbfgs', random_state = 42, max_iter=1000)

cross_val_score(log, X, y, cv=10, scoring='accuracy').mean()
# grid = GridSearchCV(log, 
#                     param_grid = param_grid, 
#                     cv = 10,
#                    scoring='accuracy')
# grid.fit(X,y)
# pd.DataFrame(grid.cv_results_)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
# knn.fit(X_train, y_train)
# y_pred = knn.predict(X_test)
# metrics.accuracy_score(y_test, y_pred)

np.float64(0.5829768270944741)

In [108]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)
cross_val_score(rfc, X, y, cv=10, scoring='accuracy').mean()

np.float64(0.5829768270944741)

In [106]:
y.value_counts(normalize = True)

AgeGroup
YoungAdult    0.522659
Adult         0.283988
Teen          0.087613
Child         0.075529
Senior        0.030211
Name: proportion, dtype: float64